# Imports

In [ ]:
# imports
import pennylane as qml
import numpy as np
import torch
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

In [ ]:
import logging
logger = logging.getLogger(__name__)
logger.setLevel(level=logging.INFO)
logger.info("hello world")

In [ ]:
import torch
print(torch.__version__)

In [ ]:
from qulearn.qlayer import IQPEmbeddingLayer
from qulearn.qlayer import QEvalType
import torch
import pennylane as qml
from qulearn.observable import parity_all_hamiltonian 

numq = 3
wires = range(numq)
qdev = qml.device("lightning.qubit", wires=numq, shots=None)
@qml.qnode(qdev)
def qnode():
    return qml.expval(qml.PauliZ(0))

out = qnode()
print(out)
print(qdev.shots)

In [ ]:
import torch
import math
t = torch.nn.Parameter(torch.zeros(3, 3))
print(t)
torch.nn.init.uniform_(t, a=0.0, b=2*math.pi)
print(t)
a = [0, 1]
b = [0, 2]
print(a==b)

In [53]:
from qulearn.qlayer import IQPEmbeddingLayer, RYCZLayer, MeasurementLayer, HamiltonianLayer, MeasurementType, EmbedVarLayer, CircuitLayer
import pennylane as qml
import torch
from qulearn.observable import parities_all_observables

observable = qml.PauliZ(0)
num_wires = 3
num_reup = 1
num_layers = 1
num_repeat = 3
qdev = qml.device('default.qubit.autograd', wires=num_wires, shots=None)

upload_layer = IQPEmbeddingLayer(num_wires, num_reup)
var_layer = RYCZLayer(num_wires, num_layers)
embedvar = EmbedVarLayer(upload_layer, var_layer, n_repeat=num_repeat, omega=1.0)
measure_layer = MeasurementLayer(embedvar, qdevice=qdev, measurement_type=MeasurementType.Expectation, observable=observable)
obs = parities_all_observables(num_wires)
ham_layer = HamiltonianLayer(embedvar,  observables=obs, interface='torch', diff_method='backprop')
#ham_layer = HamiltonianLayer(upload_layer, observables=obs)

x = torch.zeros(3, num_wires)
print(ham_layer(x))

tensor([[-0.2039],
        [-0.2039],
        [-0.2039]], dtype=torch.float64, grad_fn=<StackBackward0>)


In [26]:
for p in ham_layer.parameters():
    print(p)
print(ham_layer.observable.coeffs)
print(ham_layer.observable)

Parameter containing:
tensor([-0.2721,  0.4321, -0.1586, -0.6842, -0.1647, -0.7057,  1.1033, -1.0298],
       requires_grad=True)
Parameter containing:
tensor([3.8389, 1.4313, 3.5868], requires_grad=True)
Parameter containing:
tensor([[[1.1019, 0.7775],
         [2.8750, 2.0863]]], requires_grad=True)
Parameter containing:
tensor([-0.2721,  0.4321, -0.1586, -0.6842, -0.1647, -0.7057,  1.1033, -1.0298],
       requires_grad=True)
  (-1.0297938585281372) [I0]
+ (-0.70566725730896) [Z1]
+ (-0.684208869934082) [Z0]
+ (1.103307843208313) [Z2]
+ (-0.16468751430511475) [Z1 Z2]
+ (-0.15856914222240448) [Z0 Z2]
+ (0.4320738613605499) [Z0 Z1]
+ (-0.27214688062667847) [Z0 Z1 Z2]


In [ ]:
tmp = torch.tensor([1., -1., 0.5])
print(tmp.numel())

In [ ]:
from qulearn.fim import empirical_fim, compute_fims, mc_integrate_idfim_det

FIM = empirical_fim(measure_layer, x)
for p in measure_layer.parameters():
    print(p)

plist = []
p1 = torch.randn(4, 3)
p2 = torch.randn(4, 3, 2, 2)
for i in range(4):
    plist.append([p1[i], p2[i]])

FIMs = compute_fims(measure_layer, x, plist)

In [ ]:
integral = mc_integrate_idfim_det(FIMs, 1.0, 1.0)
print(integral)

In [ ]:
lin = torch.nn.Linear(3, 1)
print(lin(x))

In [46]:
y1 = upload_layer(x)
y2 = var_layer(x)
#y3 = measure_layer(x)
y4 = ham_layer(x)

print(y3)
print(y4)

QuantumFunctionError: Adjoint differentiation method does not support Hamiltonian observables.

In [ ]:
import torch
print(y3)
log_prob = torch.log(y3)
print(torch.log(y3))
print(x.shape)

measure_layer.zero_grad()
log_prob.backward(retain_graph=True)
grad_list = [
    p.grad.view(-1)
    for p in measure_layer.parameters()
    if p.requires_grad and p.grad is not None
]
grad = torch.cat(grad_list)
prod = torch.outer(grad, grad)

In [ ]:
for key, val in measure_layer.state_dict().items():
    print(key)
    print(val)
    print("====")

print("************************************")

for key, val in ham_layer.state_dict().items():
    print(key)
    print(val)
    print("====")

In [ ]:
from torch import nn
class HybridModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.ll_in = nn.Linear(3, 3, dtype=torch.float64)
        self.meas = ham_layer
        self.ll_out = nn.Linear(1, 1, dtype=torch.float64)

    def forward(self, x):

        y = self.ll_in(x)
        y = self.meas(y)
        y = self.ll_out(y)

        return y
    
hybrid = HybridModel()
x = torch.rand(3, dtype=torch.float64)
y = hybrid(x)
print(y)
for key, val in hybrid.state_dict().items():
    print(key)
    print(val)
    print("======")

In [ ]:
print(y)

In [ ]:
for key, val in measure_layer.state_dict().items():
    print(key)
    print(val)
    print("=====")

In [ ]:
from enum import Enum

class MyType(Enum):

    TypeA = 0
    TypeB = 1

x = MyType.TypeA
y = "blub"

if x == MyType.TypeA:
    print(x)

if not isinstance(y, MyType):
    raise NotImplementedError("QEvalType not implemented")

In [ ]:
import torch
from torch.nn import Linear
model = Linear(3, 1)
X = torch.tensor([[0.1, 1.1, -2.2], [0.6, 4.1, -3.2], [-0.1, -2.1, -2.2]])
Y = model(X)
print(Y.shape)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/test")
for i in range(10):
    val = 1e-01
    writer.add_scalar("foobar1", val, i)
    writer.add_scalars("loss", {"train": val, "valid": val+1.0}, i)
writer.close()

In [ ]:
import torch
import pennylane as qml
from qml_mor.models import IQPEReuploadSU2Parity, ModelType
from torch.utils.data import TensorDataset, DataLoader

num_samples=1
num_qubits = 3
num_reups = 1
num_layers = 1
sizex = num_qubits

omega = 0.0
init_theta = torch.randn(num_reups, num_qubits, requires_grad=True)
theta = torch.randn(
    num_reups, num_layers, num_qubits - 1, 2, requires_grad=True
)
W = torch.randn(2**num_qubits, requires_grad=True)

params = [init_theta, theta, W]

dev = qml.device("default.qubit", wires=num_qubits, shots=None)

model = IQPEReuploadSU2Parity(dev, params, model_type=ModelType.Expectation)
X = torch.randn(10, 3)
Ypred = model(X)
print(Ypred)
print(Ypred.shape)

loss = torch.nn.MSELoss()
test = loss(Ypred, Ypred)
print(test)

In [ ]:
from torch.nn import Parameter
P = model.parameters()
for p in P:
    print(p)
print("====================")
save = model.state_dict().copy()

init_theta = torch.randn(num_reups, num_qubits, requires_grad=True)
theta = torch.randn(
    num_reups, num_layers, num_qubits - 1, 2, requires_grad=True
)
W = torch.randn(2**num_qubits, requires_grad=True)

params = [init_theta, theta, W]

model = IQPEReuploadSU2Parity(dev, params)

P = model.parameters()
for p in P:
    print(p)
print("====================")
model.load_state_dict(save)
P = model.parameters()
for p in P:
    print(p)
print("====================")
print(save)

In [87]:
import os
os.environ["OMP_NUM_THREADS"] = "8"
from qulearn.qlayer import IQPEmbeddingLayer, RYCZLayer, MeasurementLayer, HamiltonianLayer, MeasurementType, EmbedVarLayer, CircuitLayer
import pennylane as qml
import torch
from qulearn.observable import parities_all_observables

observable = qml.PauliZ(0)
num_wires = 3
num_reup = 1
num_layers = 1
num_repeat = 3
qdev = qml.device('default.qubit', wires=num_wires, shots=None)

upload_layer = IQPEmbeddingLayer(num_wires, num_reup)
var_layer = RYCZLayer(num_wires, num_layers)
embedvar = EmbedVarLayer(upload_layer, var_layer, n_repeat=num_repeat, omega=1.0)
obs = parities_all_observables(num_wires)
ham_layer = HamiltonianLayer(embedvar,  qdevice=qdev, observables=obs, interface='torch', diff_method='adjoint')

x = torch.zeros(3, num_wires)
print(ham_layer(x))

QuantumFunctionError: Adjoint differentiation method does not support Hamiltonian observables.

In [109]:
import torch
from torch.utils.data import DataLoader, TensorDataset
N = 100
torch.manual_seed(0)
X = torch.randn(N, 10, dtype=torch.float64)
#Y = torch.randn(N, 1, dtype=torch.float64)
A = torch.randn(10, 1, dtype=torch.float64)
b = torch.randn(1, dtype=torch.float64)
eps = torch.randn(N, dtype=torch.float64)*0.001
Y = torch.matmul(X, A) + b + eps
model = torch.nn.Linear(10, 1, bias=True, dtype=torch.float64)
batch_size=4
shuffle=True

dataset = TensorDataset(X, Y)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

In [110]:
from torch.optim import Adam
from qulearn.trainer import SupervisedTrainer
import logging
from torch.utils.tensorboard import SummaryWriter
opt = Adam(model.parameters(), lr=0.1, amsgrad=True)
loss_fn = torch.nn.MSELoss()
metrics = {"L1": loss_fn, "L2": loss_fn}
writer = SummaryWriter()
logger = logging.getLogger("SupTrainer")
logger.setLevel(level=logging.INFO)
trainer = SupervisedTrainer(opt, loss_fn, metrics, 200, logger=logger)

In [111]:
trainer.train(model, loader, loader)

INFO:SupTrainer:Train - Epoch: 1, Loss: 0.5244, Metrics: L1: 0.5244, L2: 0.5244
INFO:SupTrainer:Validate - Epoch: 1, Loss: 0.5244, Metrics: L1: 0.5244, L2: 0.5244
INFO:SupTrainer:Train - Epoch: 2, Loss: 0.1121, Metrics: L1: 0.1121, L2: 0.1121
INFO:SupTrainer:Validate - Epoch: 2, Loss: 0.1121, Metrics: L1: 0.1121, L2: 0.1121


INFO:SupTrainer:Train - Epoch: 3, Loss: 0.0101, Metrics: L1: 0.0101, L2: 0.0101
INFO:SupTrainer:Validate - Epoch: 3, Loss: 0.0101, Metrics: L1: 0.0101, L2: 0.0101
INFO:SupTrainer:Train - Epoch: 4, Loss: 0.0007, Metrics: L1: 0.0007, L2: 0.0007
INFO:SupTrainer:Validate - Epoch: 4, Loss: 0.0007, Metrics: L1: 0.0007, L2: 0.0007
INFO:SupTrainer:Train - Epoch: 5, Loss: 0.0001, Metrics: L1: 0.0001, L2: 0.0001
INFO:SupTrainer:Validate - Epoch: 5, Loss: 0.0001, Metrics: L1: 0.0001, L2: 0.0001
INFO:SupTrainer:Train - Epoch: 6, Loss: 0.0000, Metrics: L1: 0.0000, L2: 0.0000
INFO:SupTrainer:Validate - Epoch: 6, Loss: 0.0000, Metrics: L1: 0.0000, L2: 0.0000
INFO:SupTrainer:Train - Epoch: 7, Loss: 0.0000, Metrics: L1: 0.0000, L2: 0.0000
INFO:SupTrainer:Validate - Epoch: 7, Loss: 0.0000, Metrics: L1: 0.0000, L2: 0.0000
INFO:SupTrainer:Train - Epoch: 8, Loss: 0.0000, Metrics: L1: 0.0000, L2: 0.0000
INFO:SupTrainer:Validate - Epoch: 8, Loss: 0.0000, Metrics: L1: 0.0000, L2: 0.0000
INFO:SupTrainer:Train 

In [86]:
%%timeit
trainer.train(ham_layer, loader, loader) # lightning + torch + adjoint (+ omp-num-threads=8)

INFO:SupTrainer:Train - Epoch: 1, Loss: 1.9839, Metrics: L1: 1.9839, L2: 1.9839
INFO:SupTrainer:Validate - Epoch: 1, Loss: 1.9839, Metrics: L1: 1.9839, L2: 1.9839
INFO:SupTrainer:Train - Epoch: 2, Loss: 1.3793, Metrics: L1: 1.3793, L2: 1.3793
INFO:SupTrainer:Validate - Epoch: 2, Loss: 1.3793, Metrics: L1: 1.3793, L2: 1.3793
INFO:SupTrainer:Train - Epoch: 3, Loss: 1.6594, Metrics: L1: 1.6594, L2: 1.6594
INFO:SupTrainer:Validate - Epoch: 3, Loss: 1.6594, Metrics: L1: 1.6594, L2: 1.6594
INFO:SupTrainer:Train - Epoch: 4, Loss: 1.7433, Metrics: L1: 1.7433, L2: 1.7433
INFO:SupTrainer:Validate - Epoch: 4, Loss: 1.7433, Metrics: L1: 1.7433, L2: 1.7433
INFO:SupTrainer:Train - Epoch: 5, Loss: 2.0461, Metrics: L1: 2.0461, L2: 2.0461
INFO:SupTrainer:Validate - Epoch: 5, Loss: 2.0461, Metrics: L1: 2.0461, L2: 2.0461
INFO:SupTrainer:Train - Epoch: 1, Loss: 1.5082, Metrics: L1: 1.5082, L2: 1.5082
INFO:SupTrainer:Validate - Epoch: 1, Loss: 1.5082, Metrics: L1: 1.5082, L2: 1.5082
INFO:SupTrainer:Train 

9.37 s ± 486 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [79]:
%%timeit
trainer.train(ham_layer, loader, loader) # default + torch + backprop

INFO:SupTrainer:Train - Epoch: 1, Loss: 0.8819, Metrics: L1: 0.8819, L2: 0.8819
INFO:SupTrainer:Validate - Epoch: 1, Loss: 0.8819, Metrics: L1: 0.8819, L2: 0.8819
INFO:SupTrainer:Train - Epoch: 2, Loss: 0.8601, Metrics: L1: 0.8601, L2: 0.8601
INFO:SupTrainer:Validate - Epoch: 2, Loss: 0.8601, Metrics: L1: 0.8601, L2: 0.8601
INFO:SupTrainer:Train - Epoch: 3, Loss: 0.7551, Metrics: L1: 0.7551, L2: 0.7551
INFO:SupTrainer:Validate - Epoch: 3, Loss: 0.7551, Metrics: L1: 0.7551, L2: 0.7551
INFO:SupTrainer:Train - Epoch: 4, Loss: 0.7119, Metrics: L1: 0.7119, L2: 0.7119
INFO:SupTrainer:Validate - Epoch: 4, Loss: 0.7119, Metrics: L1: 0.7119, L2: 0.7119
INFO:SupTrainer:Train - Epoch: 5, Loss: 0.7337, Metrics: L1: 0.7337, L2: 0.7337
INFO:SupTrainer:Validate - Epoch: 5, Loss: 0.7337, Metrics: L1: 0.7337, L2: 0.7337
INFO:SupTrainer:Train - Epoch: 1, Loss: 0.7237, Metrics: L1: 0.7237, L2: 0.7237
INFO:SupTrainer:Validate - Epoch: 1, Loss: 0.7237, Metrics: L1: 0.7237, L2: 0.7237
INFO:SupTrainer:Train 

38.7 s ± 210 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
trainer.train(ham_layer, loader, loader) # default + torch + adjoint

In [ ]:
%lprun -T lprof0 -u 1.0 -f trainer._train_step trainer.train(ham_layer, loader, loader)
writer.close()

In [ ]:
from enum import Enum

class MeasurementType(Enum):
    """Measurement type for a measurement layer."""

    """Expectation: return expected value of observable."""
    Expectation
    """Probabilities: return vector of probabilities."""
    Probabilities = "probabilities"
    """Samples: return measurement samples."""
    Samples = "samples"

example = MeasurementType.Expectation
print(type(example.name))

In [ ]:
from qml_mor.trainer import RegressionTrainer
from torch.utils.tensorboard import SummaryWriter
from torch.optim import Adam
from datetime import datetime

optimizer = Adam(model.parameters(), lr=0.1, amsgrad=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/check_{}'.format(timestamp))
loss_fn = torch.nn.MSELoss()
training = RegressionTrainer(optimizer, loss_fn, writer)
loss = training.train(model, loader, loader)
print(loss)
writer.close()

In [ ]:
path = "model_bestmre"
state = torch.load(path)
model.load_state_dict(state)
Ypred = model(X)
print(Y)
print("==============")
print(Ypred)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/fashion_trainer_20230517_113403

In [ ]:
import torch
import pennylane as qml
from torch.utils.data import TensorDataset, DataLoader
from qml_mor.models import IQPEReuploadSU2Parity
from qml_mor.trainer import AdamTorch
from qml_mor.datagen import DataGenCapacity

num_samples=1
num_qubits = 3
num_reups = 1
num_layers = 1
sizex = num_qubits

datagen = DataGenCapacity(sizex=sizex, num_samples=num_samples)

omega = 0.0
init_theta = torch.randn(num_reups, num_qubits, requires_grad=True)
theta = torch.randn(
    num_reups, num_layers, num_qubits - 1, 2, requires_grad=True
)
W = torch.randn(2**num_qubits, requires_grad=True)

params = [init_theta, theta, W]

model = IQPEReuploadSU2Parity(omega)
dev = qml.device("default.qubit", wires=num_qubits, shots=None)

@qml.qnode(dev, interface="torch")
def qnn_model(x, params):
    return model.qfunction(x, params)

loss_fn = torch.nn.MSELoss()
num_epochs = 300
opt = AdamTorch(params, loss_fn, num_epochs=num_epochs, amsgrad=True)

N = 10
batch_size=int("inf")
shuffle=True
data = datagen.gen_data(N)
X = data["X"]
Y = data["Y"][0]
dataset = TensorDataset(X, Y)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

opt_params = opt.optimize(qnn_model, loader)

In [ ]:
Nx = X.size(0)
y_pred = torch.stack([qnn_model(X[k], opt_params) for k in range(Nx)])
mre = torch.mean(torch.abs((Y - y_pred) / y_pred))

print(mre)
print(loss_fn(y_pred, Y))

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
X = torch.randn(5, 3)
Y = torch.randn(5)
batch_size = 2

dataset = TensorDataset(X, Y)
shuffle = True
loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

print(X)
print(Y)
print("===========")
for X_, Y_ in loader:
    print(X_.size(0))
    print(X_)
    print(Y_)
    print("***********")

In [ ]:
from qml_mor.datagen import DataGenRademacher, NormalPrior
prior = NormalPrior(3, seed=0)
radem = DataGenRademacher(prior, 2, 3, seed=None)
data = radem.gen_data(4)
print(data)

In [ ]:
import torch
import pennylane as qml
num_qubits = 3
W = torch.randn(2**num_qubits, requires_grad=False)
from qulearn.qlayer import parity_hamiltonian
H = parity_hamiltonian(num_qubits, W)
print(W)
print("======")
Z = qml.PauliZ(wires=0)
print(H.coeffs.requires_grad)
W_ = torch.nn.Parameter()
print(W_)

In [ ]:
import pennylane as qml
import torch
from qml_mor.models import IQPEReuploadSU2Parity

num_qubits  = 3
num_reps    = 2
num_layers  = 2
omega       = 1.
torch.manual_seed(0)
x           = torch.randn(num_qubits, requires_grad=False)
init_theta  = torch.randn(num_reps, num_qubits, requires_grad=True)
theta       = torch.randn(num_reps, num_layers, num_qubits-1, 2, requires_grad=True)
W           = torch.randn(2**num_qubits, requires_grad=True)

params = [init_theta, theta, W]

def iqpe_reupload_su2_parity(
    x, init_theta, theta, W, omega: float = 0.0
):
    """
    Quantum function that calculates the expectation value
    of the parity of Pauli Z operators.

    Args:
        x (Tensor): Input tensor of shape (num_qubits,)
        init_theta (Tensor): Initial rotation angles for each qubit,
            of shape (reps, num_qubits)
        theta (Tensor): Rotation angles for each layer and each qubit,
            of shape (reps, num_layers, num_qubits-1, 2)
        W (Tensor): Observable weights of shape (2^num_qubits,)
        omega (float, optional): Exponential feature scaling factor. Defaults to 0.0.

    Returns:
        QFuncOutput: Expectation value of the parity of Pauli Z operators
    """

    shape_init = init_theta.shape
    shape = theta.shape
    if len(shape_init) != 2:
        raise ValueError("Initial theta must be a 2-dim tensor")
    if len(shape) != 4:
        raise ValueError("Theta must be a 4-dim tensor")

    num_qubits = len(x)
    reps = shape_init[0]
    wires = range(num_qubits)

    for layer in range(reps):
        features = 2 ** (omega * layer) * x
        initial_layer_weights = init_theta[layer]
        weights = theta[layer]

        qml.IQPEmbedding(features=features, wires=wires)
        qml.SimplifiedTwoDesign(
            initial_layer_weights=initial_layer_weights,
            weights=weights,
            wires=wires,
        )

def iqpe_reupload_su2_meas(
    x, init_theta, theta, W, omega: float = 0.0
):
    iqpe_reupload_su2_parity(x, init_theta, theta, W, omega)
    obs = parities(len(x))
    H = qml.Hamiltonian(W, obs)

    return qml.sample()

#qnn_model = IQPEReuploadSU2Parity(params, omega=1.0)

dev = qml.device("default.qubit", wires=num_qubits, shots=None)
qnode = qml.QNode(iqpe_reupload_su2_meas, dev, interface="torch")
#drawer = qml.draw(qnode, expansion_strategy="device")
#print(drawer(x, init_theta, theta, W))
probs = qnode(x, init_theta, theta, W, omega)
probs

In [ ]:
samples = probs
bitstrings = [''.join(str(b.item()) for b in sample) for sample in samples]
bitstring_counts = {bs: bitstrings.count(bs) for bs in set(bitstrings)}
print(samples)
print(bitstrings)
print(bitstring_counts)

In [ ]:
from qml_mor.models import IQPEReuploadSU2Parity

model = IQPEReuploadSU2Parity()

qnode = qml.QNode(model.probabilities, dev, interface="torch")
probs = qnode(x, params)
print(probs)
print(model.Hamiltonian(params))

In [ ]:
H = model.Hamiltonian(params)
sum = 0.0
b = "110"

for idx, O in enumerate(H.ops):

    if not isinstance(O.name, list):
        if O.name == "Identity":
            sum += H.coeffs[idx]
        elif O.name == "PauliZ":
            i = O.wires[0]
            sign = (-1)**(int(b[-1-i]))
            sum += sign*H.coeffs[idx]
        else:
            raise ValueError("All operators must be PauliZ or Identity.")
    else:
        if not all(name=="PauliZ" for name in O.name):
            raise ValueError("All operators must be PauliZ or Identity.")
        
        sign = 1
        for w in O.wires:
            sign *= (-1)**(int(b[-1-w]))

        sum += sign*H.coeffs[idx]

In [ ]:
print(probs)
marginal = qml.math.marginal_prob(probs, axis=[0])
print(marginal)

In [ ]:
from qml_mor.models import parities

n = 3
test = parities(n)
H = qml.Hamiltonian(W, test)
print(H)
print(H.ops)

# Model Definition

In [ ]:
# feature layer
def feature_layer(x):
    num_qubits = len(x)
    qml.IQPEmbedding(x, wires=range(num_qubits))

In [ ]:
# variational layer
def variational_layer(init_theta, theta, num_qubits):
    qml.SimplifiedTwoDesign(initial_layer_weights=init_theta, weights=theta, wires=range(num_qubits))

In [ ]:
# observable / output layer
def sequence_generator(n):
    if n == 0:
        return [[]]
    else:
        sequences = []
        for sequence in sequence_generator(n-1):
            sequences.append(sequence + [n-1])
            sequences.append(sequence)
        return sequences
    
def parities(n):
    
    seq = sequence_generator(n)
    ops = []
    for par in seq:
        if par:
            tmp = qml.PauliZ(par[0])
            if len(par) > 1:
                for i in par[1:]:
                    tmp = tmp @ qml.PauliZ(i)

            ops.append(tmp)

    ops.append(qml.Identity(0))

    return ops

In [ ]:
from pennylane.templates import IQPEmbedding, SimplifiedTwoDesign
# QNN model
@qml.qnode(dev, interface='torch')
def qnn_model(x, init_theta, theta, W, omega=0.):

    num_qubits = len(x)
    shape_init = init_theta.shape
    reps       = 1
    if len(shape_init) < 1:
        init_theta = [init_theta]
        shape_init = init_theta.shape
    reps       = shape_init[0]

    shape_theta = theta.shape
    reps_       = 1
    if len(shape_theta) < 3:
        theta       = [theta]
        shape_theta = theta.shape
    reps_       = shape_theta[0]

    assert reps == reps_
    for l in range(reps):
        qml.IQPEmbedding(features=2**(omega*l)*x, wires=range(num_qubits))
        qml.SimplifiedTwoDesign(initial_layer_weights=init_theta[l], weights=theta[l], wires=range(num_qubits))

    obs = parities(num_qubits)
    H   = qml.Hamiltonian(W, obs)

    return qml.expval(H)

In [ ]:
import pennylane as qml
from pennylane.templates import IQPEmbedding

n_wires = 3
n_layers = 2

dev = qml.device("default.qubit", wires=n_wires)
@qml.qnode(dev, interface='torch')
def iqpe_circuit(features):
    for layer in range(n_layers):
        # Apply the IQPEmbedding template in a loop
        IQPEmbedding(features=features[layer], wires=range(n_wires))

    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_wires)]

features = np.random.random((n_layers, n_wires))

result1 = iqpe_circuit(features)

In [ ]:
num_qubits  = 3
num_reps    = 0
num_layers  = 0
omega       = 1.
x           = torch.randn(num_qubits, requires_grad=False)
init_theta  = torch.randn(num_reps, num_qubits, requires_grad=True)
theta       = torch.randn(num_reps, num_layers, num_qubits-1, 2, requires_grad=True)
W           = torch.randn(2**num_qubits, requires_grad=True)

result1 = qnn_model(x, init_theta, theta, W)
result2 = qnn_model(x, init_theta, theta, W)

ret = torch.stack([result1, result2])
result1.size()

In [ ]:
from scipy.stats import qmc
def generate_samples_r(d, S):
    sampler = qmc.LatinHypercube(d=d)
    r_samples = sampler.random(n=S)
    return r_samples
r = generate_samples_r(10, 2)
print(len(r[0]))

tmp = set()
rng = np.random.default_rng(seed=0)
b1 = tuple(rng.integers(0, 2, size=4))
b2 = tuple(rng.integers(0, 2, size=4))
tmp.add(b1)
tmp.add(b2)
print(tmp)
arr = np.array(list(tmp))
print(arr)
arr[0, 3]

In [ ]:
import torch

# Original list of tensors
tensor_list = [torch.tensor([1., 2., 3.], requires_grad=True),
               torch.tensor([4., 5., 6.], requires_grad=False)]

est_params = [
                        t.detach().clone().requires_grad_(t.requires_grad)
                        for t in tensor_list
                    ]
tensor_list *= 2
print(tensor_list)
print(est_params)

In [ ]:
msg = (f"Stopping early\n"
       "Loss not improving")
print(msg)

In [ ]:
import pennylane as qml
import torch
import torch.optim as optim
import torch.nn as nn

# Define the quantum device
dev = qml.device("default.qubit", wires=2)

# Define the custom model as a PennyLane QNode
@qml.qnode(dev, interface="torch")
def custom_model(x, y):
    # Apply your quantum circuit here, which uses input x and trainable parameters y
    # As an example, we'll use a simple circuit with one rotation gate parameterized by y:
    qml.RY(y[0], wires=0)
    qml.RX(x, wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(1))

# Define the dataset (x_data, y_data)
x_data = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5], dtype=torch.float32)
y_data = torch.tensor([0.5, 0.4, 0.3, 0.2, 0.1], dtype=torch.float32)

# Initialize the trainable parameters
params = torch.tensor([0.0], dtype=torch.float32, requires_grad=True)

# Set the hyperparameters
learning_rate = 0.01
epochs = 100
loss_fn = nn.MSELoss()
optimizer = optim.Adam([params], lr=learning_rate)

# Training loop
for epoch in range(epochs):
    optimizer.zero_grad()

    # Forward pass
    predictions = torch.tensor([custom_model(x, params) for x in x_data], dtype=torch.float32)
    loss = loss_fn(predictions, y_data)

    # Backward pass
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

print(f"Trained parameters: {params}")

In [ ]:
num_qubits  = 3
num_reps    = 3
num_layers  = 0
omega       = 1.
x           = torch.randn(num_qubits, requires_grad=False)
init_theta  = torch.randn(num_reps, num_qubits, requires_grad=True)
theta       = torch.randn(num_reps, num_layers, num_qubits-1, 2, requires_grad=True)
W           = torch.randn(2**num_qubits, requires_grad=True)
def qnn_model(x, init_theta, theta, W, omega=0.):

    num_qubits = len(x)
    shape_init = init_theta.shape
    reps       = 1
    if len(shape_init) < 1:
        init_theta = [init_theta]
        shape_init = init_theta.shape
    reps       = shape_init[0]

    shape_theta = theta.shape
    reps_       = 1
    if len(shape_theta) < 3:
        theta       = [theta]
        shape_theta = theta.shape
    reps_       = shape_theta[0]

    assert reps == reps_
    for l in range(reps):
        qml.IQPEmbedding(features=2**(omega*l)*x, wires=range(num_qubits))
        qml.SimplifiedTwoDesign(initial_layer_weights=init_theta[l], weights=theta[l], wires=range(num_qubits))

    obs = parities(num_qubits)
    H   = qml.Hamiltonian(W, obs)

    return qml.expval(H)

result = qnn_model(x, init_theta, theta, W)
print(result.shape())

In [ ]:
tmp = parities(3)
for x in tmp:
    print(type(x))
    print(issubclass(type(x), qml.operation.Observable))

In [ ]:
# example
num_qubits  = 3
num_reps    = 3
num_layers  = 0
omega       = 1.
x           = torch.randn(num_qubits, requires_grad=False)
init_theta  = torch.randn(num_reps, num_qubits, requires_grad=True)
theta       = torch.randn(num_reps, num_layers, num_qubits-1, 2, requires_grad=True)
W           = torch.randn(2**num_qubits, requires_grad=True)

print(qml.draw_mpl(qnn_model)(x, init_theta, theta, W, omega))

# Datasets for Capacity Estimation

In [ ]:
def gen_dataset(N, samples=10, seed=0):
    sizex   = num_qubits
    scale   = 2.
    shift   = -1.

    torch.manual_seed(seed)
    x       = scale*torch.rand(N, sizex, requires_grad=False) + shift
    y       = scale*torch.rand(samples, N, requires_grad=False) + shift

    return x, y

# Training

In [ ]:
# model specs
num_qubits  = 3
num_reps    = 2
num_layers  = 2
omega       = 1.

In [ ]:
import this

In [ ]:
# initial parameters
seed = 0
torch.manual_seed(seed)
init_theta  = torch.randn(num_reps, num_qubits, requires_grad=True)
theta       = torch.randn(num_reps, num_layers, num_qubits-1, 2, requires_grad=True)
W           = torch.randn(2**num_qubits, requires_grad=True)

In [ ]:
# loss function
def square_loss(targets, predictions):
    loss = 0
    for t, p in zip(targets, predictions):
        loss += (t - p) ** 2
    loss = loss / len(targets)
    return 0.5*loss

In [ ]:
# capacity estimation parameters
Nmin     = 1
Nmax     = 5
samples  = 10
steps    = 300
eps_stop = 1e-12

In [ ]:
summary = {}
for N in range(Nmin, Nmax):
    x, y = gen_dataset(N, samples)
    
    mre_sample = []
    for s in range(samples):
        print('===================================')
        def cost(init_theta, theta, W):
            pred = [qnn_model(x[k], init_theta, theta, W) for k in range(N)]
            loss = square_loss(y[s], pred)
            return loss
        
        # optimize
        opt = torch.optim.Adam([init_theta, theta, W], lr=0.1, amsgrad=True)
        for n in range(steps):
            opt.zero_grad()
            loss = cost(init_theta, theta, W)
            loss.backward()
            opt.step()

            if n%10 == 9 or n == steps - 1:
                print(f'{n+1}: {loss}')

            if loss <= eps_stop:
                break

        # compute prediction errors
        y_pred = torch.tensor([qnn_model(x[k], init_theta, theta, W) for k in range(N)], requires_grad=False)
        mre    = torch.mean(torch.abs((y[s]-y_pred)/y_pred))
        mre_sample.append(mre)

    mre_N = torch.mean(torch.tensor(mre_sample))
    summary[f'N = {N}'] = mre_N.item()

In [ ]:
for count, eps in enumerate(summary.values()):
    m = int(np.log2(1./eps.item()))
    C = (count+1)*m
    print(C)
print(torch.numel(init_theta))

In [ ]:
C = [-0.5, 13.1, 2]
max(C)

# Summary

In [ ]:
y_pred = torch.tensor([qnn_model(x[k], init_theta, theta, W) for k in range(N)], requires_grad=False)
mre    = torch.mean(torch.abs((y[s]-y_pred)/y_pred))
print(mre.item())

In [ ]:
# cutoff precision converted to bits of precision
cutoff = np.sqrt(eps_stop)
m      = np.log2(1./cutoff)
print(m)

In [ ]:
print(x)
print(init_theta)
print(theta)
print(W)
print(y[s])
print(y_pred)